In [10]:
using JuMP, GLPK

# Define season and circuit-specific data: for example, The 2023 Austrian Grand Prix at the Red Bull Ring
N = 25  # Total number of laps
MaxLapsPerStint = 9  # Maximum number of laps that can be performed on a stint
PitStopTime = 25  # Time wasted in a pit stop (in seconds)
MinPitStops = 1  # Minimum number of pit stops, enforced by regulations
MaxPitStops = N  # Maximum number of pit stops
baseLapTime = 68 # Projected race lap time, 5s off pole
degradation_rate = 0.25  # avg rate of lap time increase  due to tyre wear per lap (in seconds)
fuel_consumption_effect = 0.08  # avg Lap time decrease per lap due to fuel consumption (in seconds)

# Lap time calculation with degradation and fuel consumption effect
function lap_time_with_degradation_and_fuel(baseLapTime::Int, stint_lap::Int, lap_number::Int, 
                            degradation_rate::Float64, fuel_consumption_effect::Float64)::Float64
    return baseLapTime + (stint_lap - 1) * degradation_rate - (lap_number - 1) * fuel_consumption_effect
end

# Creating the model, with the GLPK as the solver
model = Model(GLPK.Optimizer)

stint_lap = ones(Int, N)
# Resetting stint lap counter when a pit stop is made is impossible due to the nature of Simplex
# An heuristic will be used to set the stint lap counter
for i in 1:N
    stint_lap[i] = i % MaxLapsPerStint + 1
     # println(stint_lap[i])
end

# Define decision variables - whether to pit or not on a certain lap
@variable(model, x[1:N-1], Bin)  # Pit stop decision variables (0 or 1)

# Objective function - minimize total race time
@objective(model, Min, sum(lap_time_with_degradation_and_fuel(baseLapTime, stint_lap[i], i, degradation_rate, fuel_consumption_effect) 
                                                    + PitStopTime * x[i] for i in 1:N-1))
# Constraints:

# Number of pit stops within the allowed range
@constraint(model, sum(x) >= MinPitStops)
@constraint(model, sum(x) <= MaxPitStops)

# Ensure that the race can be completed
for i in 1:(N-MaxLapsPerStint)
    @constraint(model, sum(x[j] for j in i:(i+MaxLapsPerStint-1)) >= 1)
end

# Ensure the first and last laps cannot be pit stops - regulations
@constraint(model, x[1] == 0)
@constraint(model, x[N-1] == 0)

# Solve
optimize!(model)

# Check for feasibility and view results
if termination_status(model) == MOI.INFEASIBLE
    println("Problem is infeasible.")
else
    println("Objective function value, race time: ", round(objective_value(model), digits=3), "s")
    println("-------------------\nOptimal pit strategy:")
    current_stint = 1
    for i in 1:N-1
        if value(x[i]) == 1
            println("Pitting on lap:  ", i)
            current_stint = 1
        else
            current_stint += 1
        end
    end

    for i in 1:N
        lap_time = lap_time_with_degradation_and_fuel(baseLapTime, stint_lap[i], i, degradation_rate, fuel_consumption_effect)
        println("Lap ", i, " time: ", round(lap_time,digits=3), "s")
    end
end


Objective function value, race time: 1683.17s
-------------------
Optimal pit strategy:
Pitting on lap:  9
Pitting on lap:  16
Lap 1 time: 68.25s
Lap 2 time: 68.42s
Lap 3 time: 68.59s
Lap 4 time: 68.76s
Lap 5 time: 68.93s
Lap 6 time: 69.1s
Lap 7 time: 69.27s
Lap 8 time: 69.44s
Lap 9 time: 67.36s
Lap 10 time: 67.53s
Lap 11 time: 67.7s
Lap 12 time: 67.87s
Lap 13 time: 68.04s
Lap 14 time: 68.21s
Lap 15 time: 68.38s
Lap 16 time: 68.55s
Lap 17 time: 68.72s
Lap 18 time: 66.64s
Lap 19 time: 66.81s
Lap 20 time: 66.98s
Lap 21 time: 67.15s
Lap 22 time: 67.32s
Lap 23 time: 67.49s
Lap 24 time: 67.66s
Lap 25 time: 67.83s
